In [1]:
import ROOT
import edm4hep
from podio import root_io
from math import sqrt

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x697c1e0
Welcome to JupyROOT 6.28/10


In [2]:
%jsroot on

In [3]:
ROOT.gInterpreter.Declare("#include <marlinutil/HelixClass_double.h>")
# ROOT.gInterpreter.Declare("#include <marlinutil/GeometryUtil.h>")

True

In [4]:
# Meh would require dd4hep
# Bz = ROOT.MarlinUtil.getBzAtOrigin()
Bz = 2
print(Bz)

2


In [5]:
# reader = root_io.Reader("E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.10.REC_edm4hep.root")
reader = root_io.Reader("E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.1000.REC_edm4hep.root")

In [6]:
events = reader.get("events")


In [7]:
# TODO: define beam_cone, lumi cal clearance whatever already up here and make a custom binning with one bin each in the dead parts
# caveat: for the underlay distribution this is not nice
h_total_theta = ROOT.TH1D("h_total_theta", ";#theta [#pi rad]", 30, 0, 1)
h_pass_theta = ROOT.TH1D("h_pass_theta", ";#theta [#pi rad]", 30, 0, 1)

In [8]:
all = 0
found = 0
for event in events:
    all += 1
    mcparticles = event.get("MCParticles")
    # search first genStat 1 electron
    for mcp in mcparticles:
        if (abs(mcp.getPDG()) == 11 and mcp.getGeneratorStatus() == 1):
            break
    mom = mcp.getMomentum()
    mom_v = ROOT.TVector3(mom.x, mom.y, mom.z)
    h_total_theta.Fill(mom_v.Theta() / ROOT.Math.Pi())

    mctruthrecolinks = event.get("SiTracksMCTruthLink")
    # meh need to do the whole logic as in my marlin pidutil
    # lets just care about tracks its simpler
    # argh this can also have more than one track per electron
    for link in mctruthrecolinks:
        if link.getSim() == mcp and link.getWeight() > 0.99:
            # print(f"found! with weight: {link.getWeight()}")
            found += 1
            h_pass_theta.Fill(mom_v.Theta() / ROOT.Math.Pi())
            break
print(f"track reco eff: {found / all}")

track reco eff: 0.936


In [10]:
eff = ROOT.TEfficiency(h_pass_theta, h_total_theta).CreateGraph()

In [11]:
u_theta = h_total_theta.Clone()
u_theta.Scale(1 / u_theta.Integral())
u_theta.SetLineColor(ROOT.kGray)
u_theta.SetMarkerColor(ROOT.kGray)
u_theta.SetFillColor(ROOT.kGray)

In [34]:
t = ROOT.TLatex()
t.SetTextSize(30)
t.SetTextFont(63)

In [39]:
c_eff = ROOT.TCanvas()
eff.SetTitle(";#theta_{e^{#pm}} [#pi rad]; e^{#pm} track reco efficiency")
eff.Draw("ap")
eff.GetYaxis().SetRangeUser(0, 1.05)
eff.GetXaxis().SetRangeUser(0, 1.0)
u_theta.Draw("same hist")
beam_cone = 0.11 / ROOT.Math.Pi()
ll = ROOT.TLine(beam_cone, 0, beam_cone, 1.05)
ll.SetLineColor(ROOT.kRed)
lr = ROOT.TLine(1 - beam_cone, 0, 1 - beam_cone, 1.05)
lr.SetLineColor(ROOT.kRed)
ll.Draw()
lr.Draw()
l = ROOT.TLegend(0.3, 0.3, 0.8, 0.5)
l.AddEntry(eff, "tracking efficiency")
l.AddEntry(ll, "detector acceptance")
l.AddEntry(u_theta, "electron distribution")
l.Draw()
t.DrawLatex(.3, .5, "e^{+}e^{-}#rightarrow e^{#pm}#nu W")
# c_eff.BuildLegend(0.3, 0.3, 0.9, 0.6, "foo")
c_eff.Draw()
c_eff.SaveAs("plots/trk_eff/eff_theta.pdf")

Info in <TCanvas::Print>: pdf file plots/trk_eff/eff_theta.pdf has been created
